## importing modules

In [169]:
import numpy as np
import pandas as pd
import os
import seaborn as sns

## Importing data

In [170]:
df = pd.read_csv('vehicles.csv', index_col=0)
print("data shape:")
print(df.shape)
print("columns name:")
print(list(df.columns))

data shape:
(426880, 25)
columns name:
['url', 'region', 'region_url', 'price', 'year', 'manufacturer', 'model', 'condition', 'cylinders', 'fuel', 'odometer', 'title_status', 'transmission', 'VIN', 'drive', 'size', 'type', 'paint_color', 'image_url', 'description', 'county', 'state', 'lat', 'long', 'posting_date']


## Data cleaning

### delete unnecessary columns that can't be used

In [171]:
useless_columns = ['url', 'region_url', 'VIN', 'image_url', 'description', 'state']
df.drop(useless_columns, axis=1, inplace=True)
print("data shape:")
print(df.shape)
print("columns name:")
print(list(df.columns))

data shape:
(426880, 19)
columns name:
['region', 'price', 'year', 'manufacturer', 'model', 'condition', 'cylinders', 'fuel', 'odometer', 'title_status', 'transmission', 'drive', 'size', 'type', 'paint_color', 'county', 'lat', 'long', 'posting_date']


### drop rows with key missing data

In [172]:
df.isnull().any(axis=0)

region          False
price           False
year             True
manufacturer     True
model            True
condition        True
cylinders        True
fuel             True
odometer         True
title_status     True
transmission     True
drive            True
size             True
type             True
paint_color      True
county           True
lat              True
long             True
posting_date     True
dtype: bool

In [173]:
temp = df.dropna(axis=0, subset=['year', 'manufacturer'])
print("Total delete", df.shape[0]-temp.shape[0], "row")
df = temp

Total delete 17652 row


### delete invalid data

In [174]:
def count_invalid_num(series):
    """
   Find invalid data, return the number of invalid data
    """
    invalid_num = series.isnull().sum()
    if series.price <= 0:
        invalid_num += 1
    if series.odometer < 0:
        invalid_num += 1
    return invalid_num

df['invalid_num'] = df.apply(count_invalid_num, axis=1)
df.head()

,region,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color,county,lat,long,posting_date,invalid_num
id,,,,,,,,,,,,,,,,,,,,
7316814884,auburn,33590,2014.0,gmc,sierra 1500 crew cab slt,good,8 cylinders,gas,57923.0,clean,other,NaN,NaN,pickup,white,NaN,32.590,-85.4800,2021-05-04T12:31:18-0500,3
7316814758,auburn,22590,2010.0,chevrolet,silverado 1500,good,8 cylinders,gas,71229.0,clean,other,NaN,NaN,pickup,blue,NaN,32.590,-85.4800,2021-05-04T12:31:08-0500,3
7316814989,auburn,39590,2020.0,chevrolet,silverado 1500 crew,good,8 cylinders,gas,19160.0,clean,other,NaN,NaN,pickup,red,NaN,32.590,-85.4800,2021-05-04T12:31:25-0500,3
7316743432,auburn,30990,2017.0,toyota,tundra double cab sr,good,8 cylinders,gas,41124.0,clean,other,NaN,NaN,pickup,red,NaN,32.590,-85.4800,2021-05-04T10:41:31-0500,3
7316356412,auburn,15000,2013.0,ford,f-150 xlt,excellent,6 cylinders,gas,128000.0,clean,automatic,rwd,full-size,truck,black,NaN,32.592,-85.5189,2021-05-03T14:02:03-0500,1


In [175]:
ori_rows = df.shape[0]
df.drop(df[df.invalid_num > 6].index, inplace=True)
after_rows = df.shape[0]
print("total deleted", ori_rows - after_rows, "row")

total deleted 42334 row


### deleting rows with price 0

In [176]:
ori_rows = df.shape[0]
df.drop(df[df['price'] == 0].index, inplace = True) 
after_rows = df.shape[0]
print("total deleted", ori_rows - after_rows, "Row")

total deleted 25693 Row


In [177]:
print("The current data shape is：", df.shape)


The current data shape is： (341201, 20)


## Data Imputation

In [178]:
#Interpolation function 

def fill_by_key(data:pd.DataFrame, tar_col:str, type:str, key:str, fill=False, default='GT'):
    if type == 'median':
        tmp = dict(data.groupby(key)[tar_col].median())
    if type == 'mode':
        tmp = dict(df.groupby(key)[tar_col].agg(lambda x: pd.Series.mode(x)))
        for (k, v) in tmp.items():
            if str(v).find('[') != -1:
                print(k, v, '->', default)
                tmp[k] = default
    if type == 'mean':
        tmp = dict(df.groupby(key)[tar_col].mean())
    if fill:
        df[tar_col] = df[tar_col].fillna(df[key].apply(lambda x: tmp.get(x)))
        df.drop(df[df[tar_col].isna()].index, inplace = True)
    else:
        return tmp
    
def fill_helper(data:pd.DataFrame, diction:dict, tar_col:str, key:str):
    df[tar_col] = df[tar_col].fillna(df[key].apply(lambda x: diction.get(x)))
    df.drop(df[df[tar_col].isna()].index, inplace = True)

### interpolating odometer values

In [179]:
fill_by_key(df, 'odometer', 'median', 'year', True, 1000)  # Imputation based on the median of the year
print("The current data shape is：", df.shape)

The current data shape is： (341201, 20)


In [180]:
df.isnull().any(axis=0)

region          False
price           False
year            False
manufacturer    False
model            True
condition        True
cylinders        True
fuel             True
odometer        False
title_status     True
transmission     True
drive            True
size             True
type             True
paint_color      True
county           True
lat              True
long             True
posting_date    False
invalid_num     False
dtype: bool

### interpolating model values

In [181]:
fill_ = fill_by_key(df, 'model', 'mode', 'manufacturer')
fill_helper(df, fill_, 'model', 'manufacturer')

land rover ['freelander' 'lr2' 'lr3'] -> GT
morgan [] -> GT


In [182]:
df[df.manufacturer=='hennessey'].model

Series([], Name: model, dtype: object)

In [183]:
df.isnull().any(axis=0)

region          False
price           False
year            False
manufacturer    False
model           False
condition        True
cylinders        True
fuel             True
odometer        False
title_status     True
transmission     True
drive            True
size             True
type             True
paint_color      True
county           True
lat              True
long             True
posting_date    False
invalid_num     False
dtype: bool

###  interpolating condition values

In [184]:
df.condition.unique()

array(['good', 'excellent', 'fair', 'like new', 'new', nan, 'salvage'],
      dtype=object)

In [185]:
def condition2int(condition):
    condition_dict = {'salvage':0, 'fair':1, 'good':2, 'excellent':3, 'like new':4, 'new':5}
    try:
        return condition_dict[condition]
    except:
        return np.nan

df['condition'] = df['condition'].apply(condition2int)

In [186]:
fill_by_key(df, 'condition', 'median', 'year', True, 2)
df.isnull().any(axis=0)

region          False
price           False
year            False
manufacturer    False
model           False
condition       False
cylinders        True
fuel             True
odometer        False
title_status     True
transmission     True
drive            True
size             True
type             True
paint_color      True
county           True
lat              True
long             True
posting_date    False
invalid_num     False
dtype: bool

### interpolating cylinders values

In [187]:
df.cylinders.unique()

array(['8 cylinders', '6 cylinders', nan, '4 cylinders', '5 cylinders',
       '10 cylinders', '3 cylinders', 'other', '12 cylinders'],
      dtype=object)

In [188]:
def cylinder2int(cylinders):
    try:
        return int(cylinders[0])
    except:
        return np.nan
df['cylinders'] = df['cylinders'].apply(cylinder2int)

In [189]:
fill_ = fill_by_key(df, 'cylinders', 'median', 'model', default=4)

In [190]:
fill_helper(df, fill_, 'cylinders', 'model')

### interpolating fuel values

In [191]:
%%capture 
fill_mode = fill_by_key(df, 'fuel', 'mode', 'model', default='gas')

In [192]:
fill_helper(df, fill_mode, 'fuel', 'model')
df.isnull().any(axis=0)

region          False
price           False
year            False
manufacturer    False
model           False
condition       False
cylinders       False
fuel            False
odometer        False
title_status     True
transmission     True
drive            True
size             True
type             True
paint_color      True
county           True
lat              True
long             True
posting_date    False
invalid_num     False
dtype: bool

### interpolating title_status values

In [193]:
df.title_status.unique()

array(['clean', 'rebuilt', 'lien', nan, 'salvage', 'missing',
       'parts only'], dtype=object)

In [194]:
%%capture 

fill_ = fill_by_key(df, 'title_status', 'mode', 'model', default='clean')
fill_helper(df, fill_, 'title_status', 'model')
df.isnull().any(axis=0)

### interpolating transmission, drive, size, paint_color values

In [195]:
df.transmission.unique()

array(['other', 'automatic', 'manual', nan], dtype=object)

In [196]:
%%capture 

fill_ = fill_by_key(df, 'transmission', 'mode', 'model', default='automatic')

In [197]:
fill_helper(df, fill_, 'transmission', 'model')

In [198]:
df.isnull().any(axis=0)

region          False
price           False
year            False
manufacturer    False
model           False
condition       False
cylinders       False
fuel            False
odometer        False
title_status    False
transmission    False
drive            True
size             True
type             True
paint_color      True
county           True
lat              True
long             True
posting_date    False
invalid_num     False
dtype: bool

In [199]:
df.drive.unique()

array([nan, 'rwd', '4wd', 'fwd'], dtype=object)

In [200]:
%%capture
fill_ = fill_by_key(df, 'drive', 'mode', 'model', default='fwd')


In [201]:
fill_helper(df, fill_, 'drive', 'model')
df.isnull().any(axis=0)

region          False
price           False
year            False
manufacturer    False
model           False
condition       False
cylinders       False
fuel            False
odometer        False
title_status    False
transmission    False
drive           False
size             True
type             True
paint_color      True
county           True
lat              True
long             True
posting_date    False
invalid_num     False
dtype: bool

In [202]:
df.paint_color.unique()

array(['white', 'blue', 'red', 'black', 'silver', 'grey', nan, 'brown',
       'yellow', 'orange', 'green', 'custom', 'purple'], dtype=object)

In [203]:
%%capture
fill_ = fill_by_key(df, 'paint_color', 'mode', 'model', default='white')

In [204]:
fill_helper(df, fill_, 'paint_color', 'model')
df.isnull().any(axis=0)

region          False
price           False
year            False
manufacturer    False
model           False
condition       False
cylinders       False
fuel            False
odometer        False
title_status    False
transmission    False
drive           False
size             True
type             True
paint_color     False
county           True
lat              True
long             True
posting_date    False
invalid_num     False
dtype: bool

In [205]:
df['size'].unique()

array([nan, 'full-size', 'mid-size', 'compact', 'sub-compact'],
      dtype=object)

In [206]:
%%capture
fill_ = fill_by_key(df, 'size', 'mode', 'model', default='mid-size')

In [207]:
fill_helper(df, fill_, 'size', 'model')
df.isnull().any(axis=0)

region          False
price           False
year            False
manufacturer    False
model           False
condition       False
cylinders       False
fuel            False
odometer        False
title_status    False
transmission    False
drive           False
size            False
type             True
paint_color     False
county           True
lat              True
long             True
posting_date    False
invalid_num     False
dtype: bool

In [208]:
df['type'].unique()

array(['pickup', 'truck', 'other', nan, 'coupe', 'SUV', 'mini-van',
       'sedan', 'hatchback', 'offroad', 'convertible', 'van', 'wagon',
       'bus'], dtype=object)

In [209]:
%%capture 
fill_ = fill_by_key(df, 'type', 'mode', 'model', default='other')

In [210]:
fill_helper(df, fill_, 'type', 'model')
df.isnull().any(axis=0)

region          False
price           False
year            False
manufacturer    False
model           False
condition       False
cylinders       False
fuel            False
odometer        False
title_status    False
transmission    False
drive           False
size            False
type            False
paint_color     False
county           True
lat              True
long             True
posting_date    False
invalid_num     False
dtype: bool

### Interpolate the mean of latitude and longitude 

In [211]:
df.long.fillna(df.long.mean(), inplace=True)
df.lat.fillna(df.lat.mean(), inplace=True)
df.isnull().any(axis=0)

region          False
price           False
year            False
manufacturer    False
model           False
condition       False
cylinders       False
fuel            False
odometer        False
title_status    False
transmission    False
drive           False
size            False
type            False
paint_color     False
county           True
lat             False
long            False
posting_date    False
invalid_num     False
dtype: bool

## regression model

In [212]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 312833 entries, 7316814884 to 7301591140
Data columns (total 20 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   region        312833 non-null  object 
 1   price         312833 non-null  int64  
 2   year          312833 non-null  float64
 3   manufacturer  312833 non-null  object 
 4   model         312833 non-null  object 
 5   condition     312833 non-null  float64
 6   cylinders     312833 non-null  float64
 7   fuel          312833 non-null  object 
 8   odometer      312833 non-null  float64
 9   title_status  312833 non-null  object 
 10  transmission  312833 non-null  object 
 11  drive         312833 non-null  object 
 12  size          312833 non-null  object 
 13  type          312833 non-null  object 
 14  paint_color   312833 non-null  object 
 15  county        0 non-null       float64
 16  lat           312833 non-null  float64
 17  long          312833 non-null  floa

In [213]:
df.condition = df.condition.apply(lambda x: int(x))
df.condition.unique()

array([2, 3, 1, 4, 5, 0])

In [214]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 312833 entries, 7316814884 to 7301591140
Data columns (total 20 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   region        312833 non-null  object 
 1   price         312833 non-null  int64  
 2   year          312833 non-null  float64
 3   manufacturer  312833 non-null  object 
 4   model         312833 non-null  object 
 5   condition     312833 non-null  int64  
 6   cylinders     312833 non-null  float64
 7   fuel          312833 non-null  object 
 8   odometer      312833 non-null  float64
 9   title_status  312833 non-null  object 
 10  transmission  312833 non-null  object 
 11  drive         312833 non-null  object 
 12  size          312833 non-null  object 
 13  type          312833 non-null  object 
 14  paint_color   312833 non-null  object 
 15  county        0 non-null       float64
 16  lat           312833 non-null  float64
 17  long          312833 non-null  floa

In [215]:
df.cylinders = df.cylinders.apply(lambda x: int(x))
df.cylinders.unique()

array([8, 6, 4, 5, 1, 3, 7])

In [216]:
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor

df.drop(columns=["region","posting_date","invalid_num","county"],inplace=True)

In [217]:
df.head()

,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color,lat,long
id,,,,,,,,,,,,,,,,
7316814884,33590,2014.0,gmc,sierra 1500 crew cab slt,2,8,gas,57923.0,clean,other,4wd,mid-size,pickup,white,32.590,-85.4800
7316814758,22590,2010.0,chevrolet,silverado 1500,2,8,gas,71229.0,clean,other,4wd,full-size,pickup,blue,32.590,-85.4800
7316814989,39590,2020.0,chevrolet,silverado 1500 crew,2,8,gas,19160.0,clean,other,4wd,mid-size,pickup,red,32.590,-85.4800
7316743432,30990,2017.0,toyota,tundra double cab sr,2,8,gas,41124.0,clean,other,4wd,full-size,pickup,red,32.590,-85.4800
7316356412,15000,2013.0,ford,f-150 xlt,3,6,gas,128000.0,clean,automatic,rwd,full-size,truck,black,32.592,-85.5189


In [218]:
X_train, X_val, y_train, y_val = train_test_split(df.loc[:, [x for x in list(df.columns) if x not in ['price', 'id']]], df.loc[:, 'price'], test_size=0.2 , random_state=2021)
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

(250266, 15) (62567, 15) (250266,) (62567,)


In [219]:
df[df["price"]==0]

,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color,lat,long
id,,,,,,,,,,,,,,,,


In [220]:
categorical_features_indices = np.where(X_train.dtypes != np.float64)[0]
model = CatBoostRegressor(iterations=1000, depth=5, cat_features=categorical_features_indices,learning_rate=0.05, logging_level='Verbose')

In [221]:
%%capture 

model.fit(X_train, y_train, plot=True , verbose=False)

## Predict

In [222]:
y_hat = model.predict(X_val)

In [223]:
import plotly.graph_objects as go
import numpy as np
N = 50
x = np.arange(0,N)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=np.array(y_hat[:N]),mode='lines+markers',name='y_hat'))
fig.add_trace(go.Scatter(x=x, y=np.array(y_val[:N]),mode='lines+markers',name='y_val'))
fig.show()

### It is found that the price predicts a negative number, and the data is modified. Here, the impact of outlier data on the model can be reflected.

In [224]:
y_train_log, y_val_log = np.log(y_train), np.log(y_val)

In [225]:
model.fit(X_train, y_train_log, plot=True, verbose=False)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [226]:
y_hat_log = model.predict(X_val)
y_hat = np.exp(y_hat_log)
N = 60
x = np.arange(0,N)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=np.array(y_hat[:N]),mode='lines+markers',name='y_hat'))
fig.add_trace(go.Scatter(x=x, y=np.array(y_val[:N]),mode='lines+markers',name='y_val'))
fig.show()

In [227]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
rmse_val = np.sqrt(mean_squared_error(y_val_log, y_hat_log))
print("RMSE of Validation is: ", rmse_val)

RMSE of Validation is:  0.7217211088960476


In [228]:
y_hat_log = model.predict(X_val)
y_hat = np.exp(y_hat_log)

In [229]:
model

In [230]:
import pickle
file_name = "cat_reg.pkl"

# save
pickle.dump(model, open(file_name, "wb"))


In [231]:
model1 = pickle.load(open(file_name, "rb"))


In [232]:
y_hat_log = model1.predict(X_val)
y_hat = np.exp(y_hat_log)

In [233]:
y_hat_log = model1.predict(X_val)
y_hat = np.exp(y_hat_log)
N = 60
x = np.arange(0,N)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=np.array(y_hat[:N]),mode='lines+markers',name='y_hat'))
fig.add_trace(go.Scatter(x=x, y=np.array(y_val[:N]),mode='lines+markers',name='y_val'))
fig.show()

In [234]:
import pickle
with open('unique_value.pickle', 'wb') as handle:
    pickle.dump(dic, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [235]:
from catboost import CatBoostClassifier

model1 = CatBoostClassifier()  
model1.load_model('model_save')

CatBoostError: catboost/libs/model/model_import_interface.h:19: Model file doesn't exist: model_save

In [ ]:
model.save_model("model_save")

In [ ]:
df.manufacturer.unique()

In [ ]:
a={}
for i in df.manufacturer.unique():
    a[i]=set(df[df["manufacturer"]==i]["model"])

In [ ]:
with open('manufacture_model.pickle', 'wb') as handle:
    pickle.dump(a, handle, protocol=pickle.HIGHEST_PROTOCOL)